Sería curioso hacer un Random Forest con cada año como una clase

Modelos a aplicar:
- Regresión lineal
- Regresion polinomica???
- Elastic-Net
- KNN
- Random Forrest para Regresion
- Red de neuronas mínima
- Deep learning con dropout

**TO DO:**
- Investigar recomendaciones, pautas y estrategias para un correcto preprocesamiento de un dataset
- Investigar todas los modelos de regresión y diferentes aproximaciones que se pueden realizar
- Decisión de todos los algoritmos vamos a aplicar
- Hacer árbol de decisión para registrar todas las posibles pruebas a realizar
- Investigar Power BI y Redes Neurales + Deep Learning

**Idea**
Sería una buena alternativa, contruir una red de neuronas con regularización dropout para que sea capaz de calcular la salida sin ciertos datos de entrada, puediendo así soportar los valores vacíos.

**Planificación Calendario**
* 27/03 - 18/04 (3 Semanas) -> Análisis y preprocesamiento
* 18/04 - 23/05 (5 Semanas) -> Modelos
* 23/05 - 13/06 (3 Semanas) -> Análisis de resultados
* 13/06 - 20/06 (1 Semana) -> Finiquitar memoria
* 20/06 ENTREGA DEL TFG
* 20/06 - 30/06 (1-2 semanas) -> Preparar presentación
* 30/06 PRESENTACIÓN TFG

# Importar Liberías

In [ ]:
import pandas as pd
import numpy as np
import sklearn 
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression

# Separación de la variable objetivo

# División en conjunto de entrenamiento y de test

# Modelos

## Regresión Lineal